In [1]:
import os
import math
import time
import emcee
import corner
import logging
import traceback
import numpy as np
import pandas as pd
from multiprocessing import Pool
from joblib import Parallel,delayed

from mcmc import lnlike
from mock_cmd import MockCMD
import matplotlib.pyplot as plt
plt.style.use("default")

# logfile in total.log

isochrones_dir = '/home/shenyueyue/Projects/Cluster/data/isocForMockCMD/'

In [2]:
# read smaple_obs
name = 'Melotte_22'
usecols = ['Gmag','G_BPmag','G_RPmag','phot_g_n_obs','phot_bp_n_obs','phot_rp_n_obs']
sample_obs = pd.read_csv("/home/shenyueyue/Projects/Cluster/data/Cantat-Gaudin_2020/%s.csv"%(name), usecols=usecols)
sample_obs = sample_obs.dropna().reset_index(drop=True)

# MCMC
n_stars = len(sample_obs)
step = (0.05, 0.2)
# parameter
theta_part = np.array([7.80,0.03]) # after round theta = (8.00, 0.0)  Dias+,2021:8.116,0.032 Cantat-Gaudin+,2020:7.89
scale = np.array([1, 1])
ndim = 2

# set up the MCMC sampler
nwalkers = 150
# define the step sizes for each parameter
# moves = [emcee.moves.StretchMove(a=step_sizes[i]) for i in range(ndim)]
# create an array of initial positions with small random perturbations
p0 = np.round((theta_part + scale * np.random.randn(nwalkers, ndim)), decimals=2)

# parallelization
with Pool(50) as pool:   
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlike, args=(n_stars, step, sample_obs), pool=pool) # , moves=moves
    # burn-in
    nburn = 5000
    start_burn = time.time()
    sampler.run_mcmc(p0, nburn, progress=True)
    end_burn = time.time()
    time_burn = end_burn - start_burn

samples = sampler.chain[:, :, :].reshape((-1, ndim))
figure = corner.corner(samples,
    labels=[r'log(age)',r'[M/H]',r'$f_b$',r'DM'],
        quantiles=[0.16, 0.5, 0.84],
        show_titles=True, 
        title_kwargs={"fontsize": 12},
        title_fmt = '.2f')
corner.overplot_points(figure,p0)

fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["log(age)", "mh"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])

axes[-1].set_xlabel("step number")

  1%|          | 50/5000 [00:48<1:06:22,  1.24it/s]/home/shenyueyue/Packages/miniconda3/envs/mcmc/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/shenyueyue/Packages/miniconda3/envs/mcmc/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/shenyueyue/Packages/miniconda3/envs/mcmc/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
 14%|█▍        | 718/5000 [07:04<39:18,  1.82it/s]  /home/shenyueyue/Packages/miniconda3/envs/mcmc/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:698: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/shenyueyue/Packages/minicon

ValueError: At least one parameter value was infinite